In [1]:
import pandas as pd
import numpy as np
from ddf_utils import ddf_reader as dr

In [2]:
dr.config.DDF_SEARCH_PATH = '/Users/semio/src/work/Gapminder/'

In [3]:
old_dataset = dr.DDF('ddf--gapminder--gapminder_world')
new_dataset = dr.DDF('ddf--gapminder--systema_globalis')
wdi = dr.DDF('ddf--world_bank--world_development_indicators')

In [4]:
def get_comp_df(indicator):
    '''get dataframes from old and new datasets, and combine them into one dataframe'''
    gw = old_dataset.get_datapoint_df(indicator)
    sg = new_dataset.get_datapoint_df(indicator)
    gw = gw.rename(columns={indicator: 'gw'})
    sg = sg.rename(columns={indicator: 'sg'})
    comp = pd.concat([gw, sg], axis=1)
    
    return comp

In [5]:
def rval(indicator):
    '''calculation of the R value, for more info see document of pandas.DataFrame.corr()'''
    comp = get_comp_df(indicator)
    return comp.corr().ix['gw', 'sg']

In [6]:
indicator = 'agricultural_land_percent_of_land_area'

In [7]:
get_comp_df(indicator).head()

gw        sg
geo time                 
abw 1961  11.11  11.11111
    1962  11.11  11.11111
    1963  11.11  11.11111
    1964  11.11  11.11111
    1965  11.11  11.11111

In [8]:
rval(indicator)  # the r value for `agricultural_land_percent_of_land_area`

0.99293955301907588

In [10]:
# Here are all WDI indicators need to compare

indicator_list = '''agricultural_land_percent_of_land_area
agriculture_percent_of_gdp
aid_received_per_person_current_us
aid_received_percent_of_gni
aid_received_total_us_inflation_adjusted
alternative_gdp_per_capita_ppp_wb
alternative_poverty_percent_below_nationally_defined_poverty
armed_forces_personnel_percent_of_labor_force
armed_forces_personnel_total
arms_exports_us_inflation_adjusted
arms_imports_us_inflation_adjusted
births_attended_by_skilled_health_staff_percent_of_total
broadband_subscribers
broadband_subscribers_per_100_people
cell_phones_per_100_people
cell_phones_total
children_out_of_school_primary
children_out_of_school_primary_female
children_out_of_school_primary_male
co2_intensity_of_economic_output_kg_co2_per_2005_ppp_of_gdp
contraceptive_use_percent_of_women_ages_15_49
debt_servicing_costs_percent_of_exports_and_net_income_from_abroad
debt_to_foreigners_by_public_and_private_percent_of_gni
electricity_use_per_person
electricity_use_total
energy_use_per_person
energy_use_total
expenditure_per_student_primary_percent_of_gdp_per_person
expenditure_per_student_secondary_percent_of_gdp_per_person
expenditure_per_student_tertiary_percent_of_gdp_per_person
exports_percent_of_gdp
exports_unit_value_index_2000100
external_debt_total_us_not_inflation_adjusted
extreme_poverty_percent_people_below_125_a_day
fixed_line_and_mobile_phone_subscribers_per_100_people
foreign_direct_investment_net_inflows_percent_of_gdp
foreign_direct_investment_net_outflows_percent_of_gdp
forest_area_sq_km
gdppercapita_growth_percent_per_year
gdppercapita_us_inflation_adjusted
gnipercapita_atlasmethod_current_us
gnipercapita_constant_2000_us
gnipercapita_ppp_current_international
high_technology_exports_percent_of_manufactured_exports
imports_percent_of_gdp
imports_unit_value_index_2000100
improved_sanitation_overall_access_percent
improved_sanitation_rural_access_percent
improved_sanitation_urban_access_percent
improved_water_source_overall_access_percent
improved_water_source_rural_access_percent
improved_water_source_urban_access_percent
income_share_of_2nd_poorest_20percent
income_share_of_2nd_richest_20percent
income_share_of_middle_20percent
income_share_of_poorest_10percent
income_share_of_poorest_20percent
income_share_of_richest_10percent
income_share_of_richest_20percent
industry_percent_of_gdp
inequality_index_gini
inflation_annual_percent
internet_users_per_100_people
internet_users_total_number
investments_percent_of_gdp
malnutrition_weight_for_age_percent_of_children_under_5
market_value_of_listed_companies_percent_of_gdp
medical_doctors_per_1000_people
merchandise_trade_percent_of_gdp
military_expenditure_percent_of_gdp
net_barter_terms_of_trade_2000_100
population_growth_annual_percent
population_in_urban_agglomerations_m_1_million_percent_of_total
poverty_percent_people_below_2_a_day
present_value_of_debt_percent_of_gni
primary_completion_rate_total_percent_of_relevant_age_group
primary_school_completion_percent_of_boys
primary_school_completion_percent_of_girls
pump_price_for_gasoline_us_per_liter
ratio_of_girls_to_boys_in_primary_and_secondary_education_perc
ratio_of_young_literate_females_to_males_percent_ages_15_24
rural_poverty_percent_rural_people_below_national_rural
services_percent_of_gdp
surface_area_sq_km
tax_revenue_percent_of_gdp
total_gdp_us_inflation_adjusted
total_gni_ppp_current_international
total_reserves_percent_of_debt_to_foreigners
trade_balance_percent_of_gdp
trade_balance_us_not_inflation_adjusted
urban_population
urban_population_growth_annual_percent
urban_population_percent_of_total
urban_poverty_percent_urban_people_below_national_urban'''

In [12]:
# overall r value for each indicator

rvalues = dict()

for i in indicator_list.split('\n'):
    #print(rval(i))
    rvalues[i] = rval(i)

In [14]:
pd.DataFrame.from_dict(rvalues, orient='index').sort_values(by=0).head(10)

,0
foreign_direct_investment_net_inflows_percent_of_gdp,0.295232
urban_poverty_percent_urban_people_below_national_urban,0.376169
foreign_direct_investment_net_outflows_percent_of_gdp,0.399264
aid_received_percent_of_gni,0.539419
alternative_poverty_percent_below_nationally_defined_poverty,0.741762
improved_water_source_urban_access_percent,0.803657
urban_population_growth_annual_percent,0.819809
improved_water_source_overall_access_percent,0.835962
rural_poverty_percent_rural_people_below_national_rural,0.839788
income_share_of_2nd_richest_20percent,0.852210


In [15]:
# calculate per geo statistics

In [16]:
def per_geo_rval(indicator):
    """calculation per geo rvalues for an indicator"""
    comp = get_comp_df(indicator)
    res = comp.groupby(level=0).apply(lambda x: x.corr(min_periods=5).ix['gw', 'sg']).sort_index()
    res.name = indicator
    return res

In [17]:
# per geo rvalue
res = []

for i in indicator_list.split('\n')[:-1]:
    res.append(per_geo_rval(i))
        
result = pd.concat(res, axis=1)

In [18]:
result.to_csv('/Users/semio/Desktop/res.csv')

In [19]:
# overall r-values for each geo

res = []

for i in indicator_list.split('\n')[:-1]:
    comp = get_comp_df(i)
    comp = comp.reset_index()
    comp['indicator'] = i
    comp = comp.set_index(['geo', 'time', 'indicator'])
    res.append(comp)
    
all_indicators_df = pd.concat(res)

In [20]:
geo_rvals = all_indicators_df.groupby(level=0).apply(lambda x: x.corr().ix['gw', 'sg'])

In [22]:
geo_rvals.sort_values().head(10)

geo
nru   -0.995431
abw    0.593955
vut    0.876868
gnq    0.904568
kir    0.905109
cod    0.914236
rus    0.916617
zwe    0.920398
zmb    0.920595
nzl    0.925578
dtype: float64

for the full result of above calculations, see https://docs.google.com/spreadsheets/d/1z6Qlmti2u-135ozclHeK9W7rZ5Rw0YGeI1fjcOPgl8E/edit#gid=945342426

In [23]:
# check some of the low r-value incators, see if there are wrong indicators imported.

In [35]:
indicator = 'foreign_direct_investment_net_inflows_percent_of_gdp'
indicator_wdi = 'bx_klt_dinv_wd_gd_zs'

rv = per_geo_rval(indicator)
comp = get_comp_df(indicator)
comp['ptc_changes'] = (comp['sg'] - comp['gw']) / comp['gw'] * 100

In [36]:
rv.sort_values().head(10)

geo
cyp    0.277886
mhl    0.310042
bfa    0.374770
nam    0.412999
gtm    0.422898
nzl    0.444908
kir    0.448741
nld    0.513457
mlt    0.576640
irl    0.619157
Name: foreign_direct_investment_net_inflows_percent_of_gdp, dtype: float64

In [41]:
rv.sort_values(ascending=False).head(10)

geo
syr    1.000000
egy    1.000000
ncl    1.000000
bol    1.000000
mng    0.999995
ita    0.999981
mus    0.999979
bhs    0.999973
tur    0.999960
lao    0.999951
Name: foreign_direct_investment_net_inflows_percent_of_gdp, dtype: float64

In [42]:
len(rv[rv > 0.9])

122

In [43]:
len(rv[rv <= 0.9])

23

In [ ]:
# seems that only small part of datapoints have big changes. 
# I think the indicator mapping is correct.
# now let's exam some of low r value geos

In [37]:
comp.ix['cyp']

,gw,sg,ptc_changes
time,,,
1975,3.695,3.69454,-0.012449
1976,5.624,5.62410,0.001778
1977,5.635,5.63486,-0.002484
1978,5.915,5.91543,0.007270
1979,5.473,5.47320,0.003654
1980,3.944,3.94434,0.008621
1981,3.751,3.75113,0.003466
1982,3.312,3.31231,0.009360
1983,3.164,3.16418,0.005689


In [38]:
comp.ix['mhl']

,gw,sg,ptc_changes
time,,,
1990,1.2740,0.00001,-99.999215
1991,1.2120,0.00001,-99.999175
1992,NaN,0.00001,NaN
1993,1.0050,0.00001,-99.999005
1994,-4.6270,0.00001,-100.000216
1995,-4.1590,0.00001,-100.000240
1996,-4.5100,0.00001,-100.000222
1997,-15.5800,0.00001,-100.000064
1998,-2.7410,0.00001,-100.000365


In [44]:
indicator = 'aid_received_percent_of_gni'
# indicator_wdi = 'bx_klt_dinv_wd_gd_zs'

rv = per_geo_rval(indicator)
comp = get_comp_df(indicator)
comp['ptc_changes'] = (comp['sg'] - comp['gw']) / comp['gw'] * 100

In [53]:
rv.sort_values(ascending=False).head(10)

geo
bra    0.997054
chl    0.994951
col    0.993834
mex    0.993323
chn    0.991999
hkg    0.988566
mlt    0.985283
tto    0.983170
tun    0.981990
slv    0.981467
Name: aid_received_percent_of_gni, dtype: float64

In [54]:
rv.sort_values().head(10)

geo
sle    0.071383
gnb    0.160951
geo    0.355393
slb    0.447468
bdi    0.502884
zwe    0.510313
gha    0.571945
com    0.574398
mar    0.577932
ecu    0.590335
Name: aid_received_percent_of_gni, dtype: float64

In [47]:
len(rv[rv <= 0.9])

51

In [48]:
len(rv[rv > 0.9])

59

In [55]:
comp.ix['gnb']

,gw,sg,ptc_changes
time,,,
1971,0.14010,0.50020,257.030692
1973,0.01119,0.05356,378.641644
1974,10.18000,60.56820,494.972495
1975,18.63000,123.71120,564.042941
1976,23.66000,135.99577,474.791927
1977,35.77000,171.70966,380.038188
1978,42.75000,186.41490,336.058246
1979,44.96000,202.93460,351.366993
1980,55.46000,186.31760,235.949513


In [34]:
# check if geo mapping is wrong in some low r-value cases

In [57]:
geo = dr.DDF('ddf--gapminder--geo_entity_domain')

In [58]:
geo_gap = geo.get_entities()['country']

In [59]:
geo_gap.ix['gnb']

gwid                                     i93
name                           Guinea-Bissau
world_6region             sub_saharan_africa
income_groups                     low_income
landlocked                         coastline
g77_and_oecd_countries                   g77
main_religion_2008                       NaN
gapminder_list                 Guinea-Bissau
alternative_1                     GuineaBiss
alternative_2                            NaN
alternative_3                            NaN
alternative_4_cdiac            Guinea Bissau
pandg                          GUINEA-BISSAU
god_id                                    GW
alt_5                                    NaN
upper_case_name                GUINEA-BISSAU
iso3166_1_alpha2                          GW
iso3166_1_alpha3                         GNB
iso3166_1_numeric                        624
iso3166_2                                NaN
unicode_region_subtag                     GW
arb1                                     NaN
arb2      

In [61]:
geo_wdi = wdi.get_entities()['country']

In [62]:
for i, v in geo_wdi.iterrows():
    if 'Guinea-Bissau' in v.values:
        print(i)

gnb


In [63]:
geo_wdi.ix['gnb']

country_code                                                                                       GNB
short_name                                                                               Guinea-Bissau
table_name                                                                               Guinea-Bissau
long_name                                                                    Republic of Guinea-Bissau
2_alpha_code                                                                                        GW
currency_unit                                                                   West African CFA franc
special_notes                                        In 2010, national accounts data for 2003-09 we...
region                                                                              Sub-Saharan Africa
income_group                                                                                Low income
wb_2_code                                                                